In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  6 11:38:36 2024

@author: 12345
"""

# Importing external libraries

import plotly.express as px
import plotly.offline as ply
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
import kaleido

# Parameters
# hedging tenor
tenor_hedged = '30Y'
tools_list = ['2Y','10Y','30Y']
cost_tools_dict = {'2Y':-0.25,'10Y':-0.5,'30Y':-0.5}

# Import Data

# Timeseries Data Collection - reading

data = pd.read_excel('dv01.xlsx')
DV01_params = pd.read_excel('Params.xlsx',sheet_name='DV01',index_col=0)
Vega_params = pd.read_excel('Params.xlsx',sheet_name='Vega Initial',index_col=0)
bps_Vega_params = pd.read_excel('Params.xlsx',sheet_name='bps Vega',index_col=0)

# estr_tenors = [1,2,3,4,5,6,7,8,9,10,11,12,15,20,30,40,50]

data = data.set_index('Date')

df_estr = data.copy(deep=True)
df_estr = df_estr[:-1]

combined_data = df_estr.copy(deep=True)

# swap rates plot

fig = px.line(df_estr,title = 'Swap Rates - BBG',
              width=1000,
              height=800,
              template="plotly_dark",
              labels={'value':'Swap Rates (in %)','date':"Date"})
fig.show()

ply.plot(fig)
###############################################################################
# Create a copy of the original data
# Calculate the daily change in swap rates
# Drop any NaN/infinite values (as this might affect the analysis)
# Describe the new dataset of returns
# Plot the daily change in swap rates as a line graph
###############################################################################

df = df_estr.copy(deep=True) #creating a dataframe copy of the swap rates
returns = (df - df.shift(1))*100 # calculating the daily change (in bps)

# Removing and replacing erratic values (if any)
returns.replace([np.inf, -np.inf], np.nan, inplace=True)
returns = returns.dropna(axis=0)

# Quick look at the new dataframe
returns

#%%

###############################################################################
# Compute the covariance matrix
# Apply the PCA function to fit and transform the data
# Fetch the PCA data for explained variance of the Principal Components
# Plotting the 'Scree Plot' to identify the 'contribution' of each principal 
# component to the variance
###############################################################################

# COMPUTING THE COVARIANCE MATRIX AND PERFORMING PCA

cov_matrix = returns.cov()

# PCA (fit & transform)

pca = PCA()
pca.fit_transform(cov_matrix)

# Explained variance

per_var = np.round(pca.explained_variance_ratio_*100,decimals=2)
labels = ['PC'+str(x) for x in range(1,len(per_var)+1)]
raw_bars = pd.DataFrame(per_var,index=labels) # quick dataframe to enable easy plotting of % variance explained by the principal components

# Plotting the graph

fig = px.bar(raw_bars[:8],
             title = '% of Explained Variance by PCs',
             width=600,
             height=500,
             labels={
                     "index": "Principal Component",
                     "value": "Percentage of Explained Variance"},
             template="plotly_dark"
             )
fig.update_layout(showlegend=False)
fig.show()


rands = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1],'PC3':pca.components_[2]}, index=cov_matrix.index)
rands


#%%
# Plotting the PCs across tenors
###############################################################################
# Plotting the principal components (1-3) across the tenors
# Interpreting the curve
# Calculating the actual values of the Principal Components
# each principal component is a linear combination of the original data and the loadings.
# We can calculate this across the entire time series by simply computing the dot product

###############################################################################




fig_pca = px.line(rands,
                  title = 'PCs across Tenors',
                  width=800,
                  height=700,
                  labels={"value":"Change in Yield","index":"Tenor"},
                  template="plotly_dark",
                  markers=True)
fig_pca.show()

# actual values of Principal Components

tas = returns.copy(deep=True)
pcas = np.dot(tas,rands)

# Storing the values in a dataframe

pca_df = pd.DataFrame(pcas,columns=['PC1','PC2','PC3'], index=tas.index)

# Combining the dataframes -- change in swap rates + PCA (will allow us to plot the data easily)
tas = tas.join(pca_df)
pca_df

# plotting the actual value of PC
fig_pca1 = px.line(tas[tas.columns[-3:]],
                   title = 'Principal Components (Actual Values)',
                   width=800,
                   height=700,
                   labels={"value":"Values","index":"Time"},
                   template='plotly_dark')
fig_pca1.show()
ply.plot(fig_pca1)

#%%
###############################################################################
# Calculating the Expected Daily Change and Residulas in Swap Rates
# So, for example, on any given day the change in 10yr swap is a given by its 
# loadings times the principal components
# Relative value opportunities can be spotted by plotting the residuals
###############################################################################


# Calculating the expected changes

expected_change = np.dot(pca_df,rands.T) #we use the transpose matrix rand.T to enable matrix multiplication
expected_changes = pd.DataFrame(expected_change,index = pca_df.index, columns=returns.columns)
expected_changes

# Calculating the residuals

df_residuals = returns - expected_change

# plotting for the last data point
last_index = df_residuals.index[-1]
fig_bar = px.bar(df_residuals.T[last_index],
                 width=900,
                 height=700,
                 title="PCA Residuals on "+str(last_index),
                 labels={"value":"Residual (in bps)","index":"Tenors"},
                 template="plotly_dark")
fig_bar.show()
ply.plot(fig_bar)


#%%

###############################################################################
# Hedging ratio
weight_portfolio = DV01_params.loc[tenor_hedged,:][0]
weight_tools =DV01_params.loc[tools_list,:]

#exposure on factors
loadings_hedged = rands.loc[tenor_hedged,:]
loadings_tool = rands.loc[tools_list,:]
exposure_hedged = weight_portfolio*loadings_hedged
exposure_tools = pd.DataFrame(index=weight_tools.index,columns=rands.columns)
for tool in tools_list:
    exposure_tools.loc[tool,:] = loadings_tool.loc[tool,:].values*weight_tools.loc[tool,].values
    



a=np.mat('1,2,3;2,4,8;9,6,3')
b=np.mat('1;1;3')
c=np.linalg.solve(a,b)








